# Improt some packages

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds


from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import collections
import logging
import os
import pathlib
import re
import string
import sys
import time

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow as tf
import numpy as np
import cv2
from imutils import paths
import matplotlib.pyplot as plt
import tensorflow as tf


from tensorflow import keras
from tensorflow.keras import layers
from keras.regularizers import l2

import tensorflow as tf
from scipy.ndimage.filters import gaussian_filter

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        # Select GPU number 1
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
2 Physical GPUs, 1 Logical GPUs


# Some hyperparameters settings
* For the image augmentation, use the setting in the block
* We think if decrease the information can get in original poison image, model can learn the how to generalize. So we use some image augumentation.

In [3]:
# Dataset hyperparameters
image_size = 224
image_channels = 3

# Algorithm hyperparameters
num_epochs = 20
batch_size = 64  # Corresponds to 200 steps per epoch
width = 256
temperature = 0.1
# Stronger augmentations for contrastive, weaker ones for supervised training
contrastive_augmentation = {"min_area": 0.75, "brightness": 0.9, "jitter": 0.5}
classification_augmentation = {"min_area": 0.75, "brightness": 0.9, "jitter": 0.5}

# Load the dataset
* In this section, we load the dataset from given .npy file.
* We bind the image and it label, and batch the dataset. 

In [4]:
x_train = np.load('/home/haowei/CS565600_Deep_Learning/DL_comp4/dataset/x_train_imagenet_unlearn.npy')
y_train = np.load('/home/haowei/CS565600_Deep_Learning/DL_comp4/dataset/y_train_imagenet.npy')
x_val = np.load('/home/haowei/CS565600_Deep_Learning/DL_comp4/dataset/x_val_imagenet.npy')
y_val = np.load('/home/haowei/CS565600_Deep_Learning/DL_comp4/dataset/y_val_imagenet.npy')

x_test = np.load('/home/haowei/CS565600_Deep_Learning/DL_comp4/dataset/x_test_imagenet.npy')
x_test = tf.image.rgb_to_grayscale(x_test)

y_train = np.argmax(y_train, axis=1)
y_val = np.argmax(y_val, axis=1)

train_dataset = tf.data.Dataset.from_tensor_slices((x_train))
train_ds = train_dataset.batch(batch_size, drop_remainder=True)

train_dataset_with_label = tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_ds_with_label = train_dataset_with_label.batch(batch_size, drop_remainder=True)

val_dataset_with_label = tf.data.Dataset.from_tensor_slices((x_val,y_val))
val_ds_with_label = val_dataset_with_label.batch(batch_size, drop_remainder=True)

test_dataset_with_label = tf.data.Dataset.from_tensor_slices(x_test)
test_ds_with_label = val_dataset_with_label.batch(batch_size, drop_remainder=True)

In [5]:
x_train = np.load('/home/haowei/CS565600_Deep_Learning/DL_comp4/dataset/x_train_imagenet_unlearn.npy')
# for idx, img in enumerate(x_train):
#     x_train[idx] = gaussian_filter(img, sigma=2)
y_train = np.load('/home/haowei/CS565600_Deep_Learning/DL_comp4/dataset/y_train_imagenet.npy')

x_val = np.load('/home/haowei/CS565600_Deep_Learning/DL_comp4/dataset/x_val_imagenet.npy')
# for idx, img in enumerate(x_val):
#     x_val[idx] = gaussian_filter(img, sigma=2)
y_val = np.load('/home/haowei/CS565600_Deep_Learning/DL_comp4/dataset/y_val_imagenet.npy')

x_test = np.load('/home/haowei/CS565600_Deep_Learning/DL_comp4/dataset/x_test_imagenet.npy')
# for idx, img in enumerate(x_test):
#     x_test[idx] = gaussian_filter(img, sigma=2)
x_test = tf.image.rgb_to_grayscale(x_test)

y_train = np.argmax(y_train, axis=1)
y_val = np.argmax(y_val, axis=1)

train_dataset = tf.data.Dataset.from_tensor_slices((x_train))
train_ds = train_dataset.batch(batch_size, drop_remainder=True)

train_dataset_with_label = tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_ds_with_label = train_dataset_with_label.batch(batch_size, drop_remainder=True)

val_dataset_with_label = tf.data.Dataset.from_tensor_slices((x_val,y_val))
val_ds_with_label = val_dataset_with_label.batch(batch_size, drop_remainder=True)

test_dataset_with_label = tf.data.Dataset.from_tensor_slices(x_test)
test_ds_with_label = val_dataset_with_label.batch(batch_size, drop_remainder=True)

# CNN model

We implemented a small ResNet model to train on the dataset. Moreover, instead of using maxout at the last layer, we used average pooling. The main reason of leveraging such feature is that we think the average pixel intensity is more curcial than the existence of a specific pattern. Further more, to prevent overfit, the model is smaller and the channel of CNN is less than a traditional ResNet.

In [6]:
class Residual(tf.keras.Model):
    """The Residual block of ResNet."""
    def __init__(self, num_channels, use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(num_channels, padding='same',
                                            kernel_size=3, strides=strides)
        self.conv2 = tf.keras.layers.Conv2D(num_channels, kernel_size=3,
                                            padding='same')
        self.conv3 = None
        if use_1x1conv:
            self.conv3 = tf.keras.layers.Conv2D(num_channels, kernel_size=1,
                                                strides=strides)
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.bn2 = tf.keras.layers.BatchNormalization()

    def call(self, X):
        Y = tf.keras.activations.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3 is not None:
            X = self.conv3(X)
        Y += X
        return tf.keras.activations.relu(Y)
    
class ResnetBlock(tf.keras.layers.Layer):
    def __init__(self, num_channels, num_residuals, first_block=False,
                 **kwargs):
        super(ResnetBlock, self).__init__(**kwargs)
        self.residual_layers = []
        for i in range(num_residuals):
            if i == 0 and not first_block:
                self.residual_layers.append(
                    Residual(num_channels, use_1x1conv=True, strides=1))
            else:
                self.residual_layers.append(Residual(num_channels))

    def call(self, X):
        for layer in self.residual_layers.layers:
            X = layer(X)
        return X

class Encoder(tf.keras.Model): # ResNet
    def __init__(self):
        super(Encoder, self).__init__()
        self.cov1 = tf.keras.layers.Conv2D(2, kernel_size=7, strides=2, padding='same')
        self.batch_norm =  tf.keras.layers.BatchNormalization()
        self.max_pool = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')
        self.res_1 = ResnetBlock(2, 2, first_block=True)
        self.res_2 = ResnetBlock(4, 2)
        self.res_3 = ResnetBlock(8, 2)
        self.res_4 = ResnetBlock(16, 2)
        self.pool_0 =  tf.keras.layers.AveragePooling2D()
        self.pool_1 =  tf.keras.layers.AveragePooling2D()
        self.pool_2 =  tf.keras.layers.AveragePooling2D()
        self.pool_3 =  tf.keras.layers.AveragePooling2D()
        
        self.fc = tf.keras.layers.Dense(1024)
        
    def call(self, x):
        x = self.cov1(x)
        x = self.batch_norm(x)
        x = tf.nn.relu(x)
        x = self.pool_0(x)
        x = self.res_1(x)
        x = self.pool_1(x)
        x = self.res_2(x)
        x = self.pool_2(x)
        x = self.res_4(x)
        x = self.pool_3(x)
        x = self.fc(x)

        return x  # (batch_size, input_seq_len, d_model)

# Check the encoder

In [7]:
noise = tf.random.uniform(shape=[64,224,224,1], minval=0.7, maxval=1.)

g = Encoder()
g(noise)

<tf.Tensor: shape=(64, 7, 7, 1024), dtype=float32, numpy=
array([[[[-2.97182729e-03,  1.19149499e-03,  2.07066443e-03, ...,
           1.70820300e-03, -1.52695668e-03,  1.52550652e-04],
         [-5.96311467e-04,  1.64727739e-04,  6.43122767e-04, ...,
           1.09278866e-04, -1.49633299e-04,  1.32753730e-05],
         [-5.44527174e-05,  9.11020106e-06,  6.94519695e-05, ...,
          -2.27480978e-05, -2.44529529e-05, -1.31653169e-05],
         ...,
         [-1.23819577e-08, -3.05741432e-09,  1.47598458e-08, ...,
          -1.99148431e-09, -7.37348804e-09, -3.36557648e-09],
         [-3.31509611e-08,  1.04255857e-08,  3.73234990e-08, ...,
           1.48882036e-08, -2.25056578e-08, -2.08709472e-09],
         [-2.29581651e-08,  1.86450961e-08,  2.67401301e-08, ...,
           1.91705798e-08, -2.08833271e-08,  9.69132241e-09]],

        [[-3.52617539e-03,  1.68864976e-03,  2.35124864e-03, ...,
           1.80546253e-03, -1.63540698e-03,  1.19459204e-04],
         [-4.58112539e-04, -2.

# Data augmentation
This block implement the augmentation in our project. We used 5 different augmentations, which are Random Flip, Random Translation, Random Zoom, GrayScale and sobel edge detection.

In [8]:
class RandomColorAffine(layers.Layer):
    def __init__(self, brightness=0, jitter=0, **kwargs):
        super().__init__(**kwargs)

        self.brightness = brightness
        self.jitter = jitter

    def call(self, images, training=True):
        if training:
            batch_size = tf.shape(images)[0]

            # Same for all colors
            brightness_scales = 1 + tf.random.uniform(
                (batch_size, 1, 1, 1), minval=-self.brightness, maxval=self.brightness
            )
            # Different for all colors
            jitter_matrices = tf.random.uniform(
                (batch_size, 1, 3, 3), minval=-self.jitter, maxval=self.jitter
            )

            color_transforms = (
                tf.eye(3, batch_shape=[batch_size, 1]) * brightness_scales
            )
            images = tf.clip_by_value(tf.matmul(images, color_transforms), 0, 1)
        return images


class Grayscale(layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, x):
        return tf.image.rgb_to_grayscale(x)

class Canny(layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, x):
#         blurred = cv2.GaussianBlur(x.numpy(), (5, 5), 0)
#         canny = cv2.Canny(blurred, 30, 150)
        return tf.image.sobel_edges(x)

# Image augmentation module
def get_augmenter(min_area, brightness, jitter):
    zoom_factor = 1.0 - tf.sqrt(min_area)
    return keras.Sequential(
        [
            keras.Input(shape=(image_size, image_size, image_channels)),
#             layers.experimental.preprocessing.RandomCrop(200, 200),
            layers.experimental.preprocessing.RandomFlip("horizontal"),
            layers.experimental.preprocessing.RandomTranslation(zoom_factor / 2, zoom_factor / 2),
            layers.experimental.preprocessing.RandomZoom((-zoom_factor, 0.0), (-zoom_factor, 0.0)),
            RandomColorAffine(brightness, jitter),
            Grayscale(),
            Canny(),
            layers.Reshape((224, 224, 2)),
        ]
    )

# Image augmentation module
def get_augmenter_val(min_area, brightness, jitter):
    zoom_factor = 1.0 - tf.sqrt(min_area)
    return keras.Sequential(
        [
            keras.Input(shape=(image_size, image_size, image_channels)),
            Grayscale(),
            Canny(),
            layers.Reshape((224, 224, 2)),
        ]
    )

# Model we used to classify
* We designed a model as below, we also used l2 norm to regularize our model.
* We had tried deeper layer, but it seemed useless to improve performance.

In [9]:
class classifier(tf.keras.Model): # ResNet
    def __init__(self):
        super(classifier, self).__init__()
        self.encoder = Encoder()
        self.encoder.trainable = True
        # self.fc3 = tf.keras.layers.Dense(1024, kernel_regularizer=l2(0.01))
        self.fc1 = tf.keras.layers.Dense(512, kernel_regularizer=l2(0.01))
        self.fc3 = tf.keras.layers.Dense(256)
        self.fc2 = tf.keras.layers.Dense(1)
        self.flatten = tf.keras.layers.Flatten()
        
    def call(self, x):
        x = self.encoder(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = tf.nn.relu(x)
        x = self.fc3(x)
        x = tf.nn.relu(x)
        x = self.fc2(x)
        return x

In [10]:
model = classifier()


# Training
* We tried Adam and SGD for our optimizer, and we find Adam had better performance.
* Used BCE loss in this case because it was a binary problem.

In [11]:
import keras.backend as K

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits = True)
optimizer = tf.keras.optimizers.Adam(beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-8)
# optimizer = tf.keras.optimizers.SGD(
#     learning_rate=0.001, momentum=0.0, nesterov=False, name='SGD'
# )

def binary_accuracy(y_true, y_pred):
    '''Calculates the mean accuracy rate across all predictions for binary
    classification problems.
    '''
    
    y_pred = tf.nn.sigmoid(y_pred)
    # print(y_pred)
    # print(y_pred.shape)
    y_true = tf.cast(y_true, dtype=tf.float32)
    return K.mean(K.equal(y_true, K.round(y_pred)))

@tf.function
def train_step_classification(img, label):
    
    with tf.GradientTape() as tape:
        logit = model(img)
        loss = cross_entropy(label, logit)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, logit

@tf.function
def test_step(img, label):
    logit = model(img)
    return binary_accuracy(label, logit)

# Checkpoint

In [12]:
checkpoint_path = "./checkpoints/l2-canny-slim"
# ./checkpoints/term_project_haowei_early_stop
ckpt = tf.train.Checkpoint(model=model,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=500)

In [13]:
# checkpoint_path_75 = "./checkpoints/l2-tune/over75"
# # ./checkpoints/term_project_haowei_early_stop
# ckpt_75 = tf.train.Checkpoint(model=model,
#                            optimizer=optimizer)

# ckpt_manager_75 = tf.train.CheckpointManager(ckpt_75, checkpoint_path, max_to_keep=200)

In [14]:
# ckpt.restore(checkpoint_path + '/ckpt-1313')

# Training process

In [15]:
for epoch in range(1500):
    start = time.time()
    loss = 0
    train_acc = 0
    accuracy = 0
    cnt = 1
    cnt_train = 1
    ckpt_save_path = ckpt_manager.save()
    print(f'Saving checkpoint for epoch {epoch} at {ckpt_save_path}')
    
    for batch, data in enumerate(train_ds_with_label):
        loss, logit = train_step_classification(get_augmenter(**contrastive_augmentation)(data[0]), data[1])
        logit = np.squeeze(logit)
        # print(tf.image.rgb_to_grayscale(get_augmenter(**contrastive_augmentation)(data[0])))
        train_acc += binary_accuracy(data[1], logit).numpy()
        cnt_train += 1
        
        if batch % 50 == 0:
            print(f'Epoch {epoch + 1} Batch {batch} Loss {loss:.7f}')
    for img, label in val_ds_with_label:
        accuracy += test_step(get_augmenter_val(**contrastive_augmentation)(img), label).numpy()
        cnt+=1
        
    print(f'Epoch {epoch + 1} Loss {loss:.7f}')
    print('test accuracy: ', accuracy/cnt)
    print('train accuracy: ', train_acc/cnt_train)

    print(f'Time taken for 1 epoch: {time.time() - start:.2f} secs\n')

Saving checkpoint for epoch 0 at ./checkpoints/l2-canny-slim/ckpt-1
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Epoch 1 Batch 0 Loss 0.6884665
Epoch 1 Loss 0.6551462
test accuracy:  0.6707356770833334
train accuracy:  0.6883928571428571
Time taken for 1 epoch: 8.17 secs

Saving checkpoint for epoch 1 at ./checkpoints/l2-canny-slim/ckpt-2
Epoch 2 Batch 0 Loss 0.5733029
Epoch 2 Loss 0.6174582
test accuracy:  0.70703125
train accuracy:  0.7227

ResourceExhaustedError: checkpoints/l2-canny-slim/ckpt-50_temp/part-00000-of-00001.data-00000-of-00001.tempstate7244357818491759972; No space left on device [Op:SaveV2]

# Reload model with highest validation accuracy and testing

In [16]:
ckpt.restore(checkpoint_path + '/ckpt-27')

In [17]:
import csv

header = ['id', 'label']
x_test = np.load('/home/haowei/CS565600_Deep_Learning/DL_comp4/dataset/x_test_imagenet.npy')
print(x_test.shape)
x_test = tf.image.rgb_to_grayscale(x_test)
x_test = tf.image.sobel_edges(x_test)
print(x_test.shape)
x_test = tf.reshape(x_test, [100, 224, 224, 2])
test_dataset = tf.data.Dataset.from_tensor_slices(x_test)
test_dataset = test_dataset.batch(64, drop_remainder=False)
res = np.squeeze(K.round(tf.nn.sigmoid(model.predict(test_dataset))).numpy()).astype(int)

print(res)
with open('res-canny-slim.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    
    for i, y in enumerate(res):
        writer.writerow([i, y])

(100, 224, 224, 3)
(100, 224, 224, 1, 2)
[1 0 1 1 1 0 0 1 1 0 0 1 1 0 0 0 1 0 0 1 1 0 0 0 1 0 1 0 1 0 0 1 0 1 0 1 0
 1 1 0 0 0 1 0 1 0 1 1 1 0 1 0 0 0 1 1 0 0 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 1
 0 1 0 0 0 1 1 0 1 1 1 0 1 0 0 0 0 0 0 1 1 1 0 0 0 1]


# Describe what you have done to improve your training accuracy in detail.
* We used the model architecture in above for the baseline, and try to tune the hyperparameter and do different image augmentation to improve our testing performance.
* For hyperparameter tuning, we tried different combination of epsilon and learning rate while using Adam as optimizer. We tried epsilon=1e-9, 1e-8, 1e-7; lr=1e-3, 1e-2. And we observed the validation loss, finally, we choose epsilon=1e-8 and lr=1e-3.
* For image augmentation, we noticed that when we decrease the information in the poison image which the model learned, we can let our model more general. So, beside some basic augmentation skill, we also tried "gray scale" and "sobel edge" to make our image more general for machine to identify.
* Because it is a binary problem, we thought if we just evaluate our model by public testing performance, it would overfit, so, we add l2 norm as regularization. 